In [ ]:
import numpy as np 
import cv2 
import pandas as pd
import glob
import time 
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image


def extract_sift_des(image):
    sift = cv2.ORB_create()
    kp, des = sift.detectAndCompute(image, None)
    return des

def read_bounding_box(path):
    boundary_file = open(path, 'r')
    boundary = boundary_file.readline().strip().split(' ')
    boundary = [int(b) for b in boundary]
    boundary_file.close()
    return boundary

transform = transforms.Compose([
    transforms.Resize(260),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [ ]:
download_path='C:/Users/IV/Desktop/CityU' 
path_query=download_path+'/query_4186'
path_query_txt=download_path+'/query_txt_4186'

path_gallery=download_path+'/gallery_4186'

name_query=glob.glob(path_query+'/*.jpg')
num_query=len(name_query)
name_gallery=glob.glob(path_gallery+'/*.jpg')
num_gallery=len(name_gallery)

model = models.efficientnet_b3(pretrained=True)
feat_extractor = model.features #define the feature extractor
layer1 = model.features[:-1]
feat_extractor.eval()  #set the mode as evaluation

record_all=np.zeros((num_query,len(name_gallery)))

query_imgs_no = [x.split('\\')[-1] for x in glob.glob(path_query+'/*.jpg')]
query_imgs_no = [x[:-4] for x in query_imgs_no]

gallery_imgs_no = [x.split('\\')[-1] for x in glob.glob(path_gallery+'/*.jpg')]
gallery_imgs_no = [x[:-4] for x in gallery_imgs_no]

gallery_features = []
gallery_des = []

for i, gallery_img_no in tqdm(enumerate(gallery_imgs_no)):
    per_gallery_name = path_gallery+'/'+str(gallery_img_no)+'.jpg'
    per_gallery=cv2.imread(per_gallery_name)
    
    # Image pre-processing
    per_gallery = cv2.cvtColor(per_gallery,cv2.COLOR_BGR2YUV)
    per_gallery[:,:,0] = cv2.equalizeHist(per_gallery[:,:,0])
    per_gallery = cv2.cvtColor(per_gallery,cv2.COLOR_YUV2BGR)
    per_gallery = cv2.GaussianBlur(per_gallery,(3,3),0)
    gallery_sift = extract_sift_des(per_gallery)
    per_gallery = Image.fromarray(per_gallery)

    # preprocess the input image
    img_transform = transform(per_gallery) #normalize the input image and transform it to tensor.
    img_transform = torch.unsqueeze(img_transform, 0)
    
    # feature extraction for per gallery
    with torch.no_grad():
        per_gallery_features = model(img_transform)

    gallery_features.append(per_gallery_features)
    gallery_des.append(gallery_sift)



In [ ]:
query_features = []
query_des = []
for i, query_img_no in tqdm(enumerate(query_imgs_no[0:20])):
    per_query_name=path_query+'/'+str(query_img_no)+'.jpg'
    per_query=cv2.imread(per_query_name)
    
    # read boundary from text file
    queryfilename = path_query_txt+'/'+str(query_img_no)+'.txt'
    
    # crop the image
    boundary = read_bounding_box(queryfilename)
    x ,y, w, h = boundary
    query_boundary = per_query[y:y+h, x:x+w]

    # Image pre-processing
    query_boundary=cv2.cvtColor(query_boundary,cv2.COLOR_BGR2YUV)
    query_boundary[:,:,0] = cv2.equalizeHist(query_boundary[:,:,0])
    query_boundary = cv2.cvtColor(query_boundary,cv2.COLOR_YUV2BGR)
    query_boundary = cv2.GaussianBlur(query_boundary,(3,3),0)

    query_des.append(extract_sift_des(query_boundary))
    query_boundary = Image.fromarray(query_boundary)
    query_transformed = transform(query_boundary) #normalize the input image and transform it to tensor.
    query_transformed = torch.unsqueeze(query_transformed, 0) 

    with torch.no_grad():
        query_features.append(model(query_transformed))

In [ ]:
for i, query_img_no in tqdm(enumerate(query_imgs_no[0:20])):
    time_s = time.time()
    dist_record=[]
    gallery_imgs_no_desc=[]
    # the iteration loop for gallery
    for j, gallery_img_no in tqdm(enumerate(gallery_imgs_no), desc=f"Processing query part {i}"):
        sim_score1 = cosine_similarity(query_features[i], gallery_features[j]) 
        sim_score2 = cosine_similarity(query_des[i], gallery_des[j])
        sim_score = sim_score1*0.8 + sim_score2*0.2
        dist_record.append(sim_score)
        
        # find the indexes with descending similarity order
        
    descend_index=sorted(range(len(dist_record)), key=lambda k: np.max(dist_record[k]),reverse=True)
    # update the results for one query
    for k in range(len(descend_index)):
        gallery_imgs_no_desc.append(np.array(gallery_imgs_no)[descend_index[k]])
    record_all[i,:]= gallery_imgs_no_desc
    time_e = time.time()
    print('retrieval time for query {} is {}s'.format(query_img_no, time_e-time_s))
    query_idx = i
    print(f'For query image No. {query_imgs_no[query_idx]}, the top 10 ranked similar image No. are {gallery_imgs_no_desc[0]} {gallery_imgs_no_desc[1]} {gallery_imgs_no_desc[2]} {gallery_imgs_no_desc[3]} {gallery_imgs_no_desc[4]} {gallery_imgs_no_desc[5]} {gallery_imgs_no_desc[6]} {gallery_imgs_no_desc[7]} {gallery_imgs_no_desc[8]} {gallery_imgs_no_desc[9] }')

: 

In [ ]:
# write the output file following the example
f=open(r'./rank_list_CNN&SIFT.txt','w')
for i in range(num_query):
    f.write('Q'+str(i+1)+': ')
    for j in range(len(name_gallery)):
        f.write(str(np.int32(record_all[i,j]))+' ')
    f.write('\n')
f.close()
